In [19]:
# setup
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd

RAW_PATH = Path(r"c:\Users\gence\OneDrive\Desktop\Data Governance\raw_credit_applications.json")
OUT_DIR = Path(r"c:\Users\gence\dego-project-team9\data\processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Raw file exists:", RAW_PATH.exists())


Raw file exists: True


In [20]:
# load raw + flatten main application table
records = json.loads(RAW_PATH.read_text(encoding="utf-8"))

apps_raw = pd.json_normalize(records, sep=".")
apps_raw["source_row"] = np.arange(len(apps_raw))

print("Applications rows:", len(apps_raw))
apps_raw.head(3)


Applications rows: 502


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes,source_row
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN,0
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN,1
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN,2


In [21]:
# flatten spending behavior to long table
spending_raw = pd.json_normalize(
    records,
    record_path="spending_behavior",
    meta=["_id"],
    errors="ignore",
).rename(columns={"_id": "application_id"})

print("Spending rows:", len(spending_raw))
spending_raw.head(3)


Spending rows: 827


,category,amount,application_id
0,Shopping,480,app_200
1,Rent,790,app_200
2,Alcohol,247,app_200


In [22]:
# quality profiling report
email_re = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")

def is_empty(s: pd.Series) -> pd.Series:
    return s.isna() | s.astype("string").str.strip().eq("")

def add_issue(rows, issue_type, field, count, total):
    rows.append({
        "issue_type": issue_type,
        "field": field,
        "count": int(count),
        "pct_of_rows": round(100 * int(count) / total, 2),
    })

total = len(apps_raw)
rows = []

# Duplicates
add_issue(rows, "consistency", "_id duplicate", apps_raw.duplicated("_id").sum(), total)

# Missing
for col in [
    "applicant_info.email",
    "applicant_info.ssn",
    "applicant_info.ip_address",
    "applicant_info.gender",
    "applicant_info.date_of_birth",
    "applicant_info.zip_code",
    "financials.annual_income",
    "processing_timestamp",
]:
    add_issue(rows, "completeness", f"{col} missing/blank", is_empty(apps_raw[col]).sum(), total)

# Type mismatches
income_is_str = apps_raw["financials.annual_income"].map(lambda x: isinstance(x, str)).fillna(False)
add_issue(rows, "consistency", "financials.annual_income stored as string", income_is_str.sum(), total)

# Inconsistent coding
gender = apps_raw["applicant_info.gender"].astype("string").str.strip()
gender_short_or_blank = gender.isin(["M", "F", ""]) | gender.isna()
add_issue(rows, "consistency", "applicant_info.gender inconsistent coding", gender_short_or_blank.sum(), total)

# DOB format inconsistency
dob = apps_raw["applicant_info.date_of_birth"].astype("string").str.strip()
fmt_iso = dob.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)
fmt_slash_dmy_mdy = dob.str.match(r"^\d{2}/\d{2}/\d{4}$", na=False)
fmt_slash_ymd = dob.str.match(r"^\d{4}/\d{2}/\d{2}$", na=False)
dob_non_empty = ~is_empty(dob)
dob_inconsistent = dob_non_empty & (fmt_slash_dmy_mdy | fmt_slash_ymd | (~fmt_iso & ~fmt_slash_dmy_mdy & ~fmt_slash_ymd))
add_issue(rows, "consistency", "applicant_info.date_of_birth non-ISO format", dob_inconsistent.sum(), total)

# Invalid email
email = apps_raw["applicant_info.email"].astype("string").str.strip()
invalid_email = (~is_empty(email)) & (~email.map(lambda x: bool(email_re.match(str(x)))))
add_issue(rows, "validity", "applicant_info.email invalid format", invalid_email.sum(), total)

# Invalid numeric values
credit_hist = pd.to_numeric(apps_raw["financials.credit_history_months"], errors="coerce")
dti = pd.to_numeric(apps_raw["financials.debt_to_income"], errors="coerce")
savings = pd.to_numeric(apps_raw["financials.savings_balance"], errors="coerce")

add_issue(rows, "validity", "financials.credit_history_months < 0", (credit_hist < 0).sum(), total)
add_issue(rows, "validity", "financials.debt_to_income > 1", (dti > 1).sum(), total)
add_issue(rows, "validity", "financials.savings_balance < 0", (savings < 0).sum(), total)

report_df = pd.DataFrame(rows).sort_values(["issue_type", "count"], ascending=[True, False])
report_df


,issue_type,field,count,pct_of_rows
8,completeness,processing_timestamp missing/blank,440,87.65
1,completeness,applicant_info.email missing/blank,7,1.39
2,completeness,applicant_info.ssn missing/blank,5,1.00
3,completeness,applicant_info.ip_address missing/blank,5,1.00
5,completeness,applicant_info.date_of_birth missing/blank,5,1.00
7,completeness,financials.annual_income missing/blank,5,1.00
4,completeness,applicant_info.gender missing/blank,3,0.60
6,completeness,applicant_info.zip_code missing/blank,2,0.40
11,consistency,applicant_info.date_of_birth non-ISO format,157,31.27
10,consistency,applicant_info.gender inconsistent coding,114,22.71


In [30]:
# deterministic cleaning
apps_clean = apps_raw.copy()

# Gender normalization
gender_map = {"M": "Male", "Male": "Male", "F": "Female", "Female": "Female"}
apps_clean["applicant_info.gender_clean"] = (
    apps_clean["applicant_info.gender"]
    .astype("string")
    .str.strip()
    .map(gender_map)
    .fillna("Unknown")
)

# DOB normalization to YYYY-MM-DD
# Preserve already-ISO values; parse only non-ISO values.
dob_raw = apps_clean["applicant_info.date_of_birth"].astype("string").str.strip()
iso_mask = dob_raw.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)

dob_parsed = pd.Series(pd.NaT, index=apps_clean.index, dtype="datetime64[ns]")
dob_parsed.loc[iso_mask] = pd.to_datetime(
    dob_raw[iso_mask], errors="coerce", format="%Y-%m-%d"
)
dob_parsed.loc[~iso_mask] = pd.to_datetime(
    dob_raw[~iso_mask], errors="coerce", dayfirst=True, format="mixed"
)

apps_clean["applicant_info.date_of_birth_clean"] = dob_parsed.dt.strftime("%Y-%m-%d")

# Numeric coercion
apps_clean["financials.annual_income_clean"] = pd.to_numeric(apps_clean["financials.annual_income"], errors="coerce")
apps_clean["financials.credit_history_months_clean"] = pd.to_numeric(apps_clean["financials.credit_history_months"], errors="coerce")
apps_clean["financials.debt_to_income_clean"] = pd.to_numeric(apps_clean["financials.debt_to_income"], errors="coerce")
apps_clean["financials.savings_balance_clean"] = pd.to_numeric(apps_clean["financials.savings_balance"], errors="coerce")

# Flags (transparent governance)
apps_clean["flag_duplicate_id"] = apps_clean.duplicated("_id", keep=False)
apps_clean["flag_invalid_email"] = invalid_email
apps_clean["flag_non_iso_dob_input"] = dob_inconsistent
apps_clean["flag_negative_credit_history"] = apps_clean["financials.credit_history_months_clean"] < 0
apps_clean["flag_dti_gt_1"] = apps_clean["financials.debt_to_income_clean"] > 1
apps_clean["flag_negative_savings"] = apps_clean["financials.savings_balance_clean"] < 0

print("Row number:", len(apps_clean))
apps_clean.head(3)



Row number: 502


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.annual_income_clean,financials.credit_history_months_clean,financials.debt_to_income_clean,financials.savings_balance_clean,flag_duplicate_id,flag_invalid_email,flag_non_iso_dob_input,flag_negative_credit_history,flag_dti_gt_1,flag_negative_savings
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,73000.0,23,0.20,31212,False,False,False,False,False,False
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,78000.0,51,0.18,17915,False,False,False,False,False,False
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,61000.0,41,0.21,37909,False,False,False,False,False,False


In [24]:
# Cell 6: clean spending table
spending_clean = spending_raw.copy()
spending_clean["category"] = spending_clean["category"].astype("string").str.strip()
spending_clean["amount"] = pd.to_numeric(spending_clean["amount"], errors="coerce")


print("Row number:", len(spending_clean))

spending_clean.head(3)

Row number: 827


,category,amount,application_id
0,Shopping,480,app_200
1,Rent,790,app_200
2,Alcohol,247,app_200


In [31]:
# cleaning log + save outputs
cleaning_log = pd.DataFrame([
    {"field": "applicant_info.gender", "rule": "Normalize M/F/Male/Female to Male/Female; else Unknown",
     "rows_affected": int(gender_short_or_blank.sum())},
    {"field": "applicant_info.date_of_birth", "rule": "Parse mixed formats and standardize to YYYY-MM-DD",
     "rows_affected": int(dob_inconsistent.sum())},
    {"field": "financials.annual_income", "rule": "Coerce to numeric",
     "rows_affected": int(income_is_str.sum())},
    {"field": "_id", "rule": "Flag duplicate application IDs",
     "rows_affected": int(apps_clean["flag_duplicate_id"].sum())},
])

apps_out = OUT_DIR / "applications_clean.csv"
spending_out = OUT_DIR / "spending_clean.csv"
report_out = OUT_DIR / "data_quality_report.csv"
log_out = OUT_DIR / "cleaning_log.csv"

apps_clean.to_csv(apps_out, index=False)
spending_clean.to_csv(spending_out, index=False)
report_df.to_csv(report_out, index=False)
cleaning_log.to_csv(log_out, index=False)

print("Saved:", apps_out)
print("Saved:", spending_out)
print("Saved:", report_out)
print("Saved:", log_out)

Saved: c:\Users\gence\dego-project-team9\data\processed\applications_clean.csv
Saved: c:\Users\gence\dego-project-team9\data\processed\spending_clean.csv
Saved: c:\Users\gence\dego-project-team9\data\processed\data_quality_report.csv
Saved: c:\Users\gence\dego-project-team9\data\processed\cleaning_log.csv


In [26]:
# post-cleaning validation summary
flag_cols = [col for col in apps_clean.columns if col.startswith("flag_")]
apps_clean["any_quality_flag"] = apps_clean[flag_cols].any(axis=1)

total_rows = len(apps_clean)
flagged_rows = apps_clean["any_quality_flag"].sum()
clean_rows = total_rows - flagged_rows

print("\n=== POST-CLEANING DATA VALIDATION ===")
print(f"Total application records: {total_rows:,}")
print(f"Records with ≥1 quality flag: {flagged_rows:,} ({100*flagged_rows/total_rows:.1f}%)")
print(f"Records with no flags: {clean_rows:,} ({100*clean_rows/total_rows:.1f}%)")
print(f"\nFlag summary:")
for col in sorted(flag_cols):
    count = apps_clean[col].sum()
    pct = 100 * count / total_rows
    print(f"  {col}: {count:,} ({pct:.1f}%)")

print(f"\n✓ Data retention strategy: ALL {total_rows:,} rows retained")
print(f"✓ Cleaned fields available with '_clean' or descriptive suffix")
print(f"✓ Original values preserved for audit trail")
print(f"✓ Ready for bias analysis with optional flag-based filtering")


=== POST-CLEANING DATA VALIDATION ===
Total application records: 502
Records with ≥1 quality flag: 169 (33.7%)
Records with no flags: 333 (66.3%)

Flag summary:
  flag_dti_gt_1: 1 (0.2%)
  flag_duplicate_id: 4 (0.8%)
  flag_invalid_email: 4 (0.8%)
  flag_negative_credit_history: 2 (0.4%)
  flag_negative_savings: 1 (0.2%)
  flag_non_iso_dob_input: 157 (31.3%)

✓ Data retention strategy: ALL 502 rows retained
✓ Cleaned fields available with '_clean' or descriptive suffix
✓ Original values preserved for audit trail
✓ Ready for bias analysis with optional flag-based filtering


In [32]:
# expanded cleaning log with all transformations
cleaning_log_extended = pd.DataFrame([
    {"field": "applicant_info.gender", "rule": "Normalize M/F/Male/Female to Male/Female/Unknown", 
     "rows_affected": int(gender_short_or_blank.sum()), "handling": "Retain with gender_clean field"},
    {"field": "applicant_info.date_of_birth", "rule": "Parse mixed formats to YYYY-MM-DD", 
     "rows_affected": int(dob_inconsistent.sum()), "handling": "Retain; flag non-ISO inputs"},
    {"field": "financials.annual_income", "rule": "Coerce string values to numeric", 
     "rows_affected": int(income_is_str.sum()), "handling": "Retain with income_clean field"},
    {"field": "_id", "rule": "Identify duplicate IDs", 
     "rows_affected": int(apps_clean["flag_duplicate_id"].sum()), "handling": "Retain; flag for review"},
    {"field": "applicant_info.email", "rule": "Validate email format (RFC-like pattern)", 
     "rows_affected": int(invalid_email.sum()), "handling": "Retain; flag invalid emails"},
    {"field": "financials.credit_history_months", "rule": "Coerce to numeric; identify negative values", 
     "rows_affected": int((apps_clean["financials.credit_history_months_clean"] < 0).sum()), "handling": "Retain; flag negative values"},
    {"field": "financials.debt_to_income", "rule": "Coerce to numeric; identify DTI > 1", 
     "rows_affected": int((apps_clean["financials.debt_to_income_clean"] > 1).sum()), "handling": "Retain; flag invalid DTI"},
    {"field": "financials.savings_balance", "rule": "Coerce to numeric; identify negative values", 
     "rows_affected": int((apps_clean["financials.savings_balance_clean"] < 0).sum()), "handling": "Retain; flag negative balance"},
])

print("\n=== COMPREHENSIVE CLEANING LOG ===")
print(cleaning_log_extended.to_string(index=False))


=== COMPREHENSIVE CLEANING LOG ===
                           field                                             rule  rows_affected                       handling
           applicant_info.gender Normalize M/F/Male/Female to Male/Female/Unknown            114 Retain with gender_clean field
    applicant_info.date_of_birth                Parse mixed formats to YYYY-MM-DD            157    Retain; flag non-ISO inputs
        financials.annual_income                  Coerce string values to numeric              8 Retain with income_clean field
                             _id                           Identify duplicate IDs              4        Retain; flag for review
            applicant_info.email         Validate email format (RFC-like pattern)              4    Retain; flag invalid emails
financials.credit_history_months      Coerce to numeric; identify negative values              2   Retain; flag negative values
       financials.debt_to_income              Coerce to numeric; ide

## Data Retention Strategy

All rows are retained in the cleaned dataset with quality flags. This approach:
- Preserves data for transparency and audit trails
- Allows downstream analysis to handle questionable records
- Enables sensitivity analysis by including/excluding flagged rows
- Documents decision points for reproducibility

Flagged fields are stored with "_clean" suffix; original values preserved for reference.